In [1]:
import googlemaps
import folium
from haversine import haversine
from sklearn import preprocessing, cluster
from sklearn.metrics.pairwise import haversine_distances

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore") #경고 표시 제거는 일시적으로만 하자... 중요한 내용일수도

import matplotlib.font_manager as fm #한글 안깨지게
# font_name=fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
# plt.rc("font", family=font_name)

import matplotlib as mpl # - 표시 정상적으로 되도록
mpl.rcParams["axes.unicode_minus"]=False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
import time

In [3]:
df = pd.read_csv("data/맛집리스트_위도경도분리(0108).csv", sep=",", index_col=0)
df2 = pd.read_csv("data/좌표 중복값 제거 리스트.csv", sep=",")

In [4]:
df3=df2.sort_values(by="구").reset_index()
df3.drop(["index"], axis=1, inplace=True)

In [5]:
coords = df3[["위도", "경도"]].to_numpy()

In [6]:
# DBSCAN으로 Cluster 구하기 --> 최대 확장거리 200m, 10개 이상 모여있을 시 Cluster로 간주

kms_per_radian = 6371.0088
epsilon = 0.2 / kms_per_radian #거리를 km 단위로 환산, 
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
print(db)
print("-"*80)
cluster_labels = db.labels_
print(cluster_labels)
print("-"*80)
num_clusters = len(set(cluster_labels))
print(set(cluster_labels))
print("-"*80)
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print(clusters)
print("-"*80)
print('Number of clusters: {}'.format(num_clusters))

DBSCAN(algorithm='ball_tree', eps=3.139220275445233e-05, metric='haversine',
       min_samples=10)
--------------------------------------------------------------------------------
[-1  0  0 ... -1 -1 -1]
--------------------------------------------------------------------------------
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, -1}
--------------------------------------------------------------------------------
0     [[37.5260204, 127.037079], [37.5258965, 127.03...
1     [[37.5181233, 127.025218], [37.5217787, 127.02...
2     [[37.527649, 127.1248143], [37.5285963, 127.12...
3     [[37.538851, 127.124463], [37.5391443, 127.127...
4     [[37.52316800000001, 127.1311612], [37.5244541...
                            ...                        
65    [[37.5995475, 127.09

In [7]:
# 각 Cluster의 중앙 좌표 계산

centroid=[]

for i in range(len(clusters)-1):
    lan=[]
    lon=[]
    for j in range(len(clusters[i])):
        lan.append(clusters[i][j][0])
        lon.append(clusters[i][j][1])
    centroid.append((sum(lan)/len(clusters[i]), sum(lon)/len(clusters[i])))

In [8]:
# 각 cluster의 중앙 좌표만 찍어보기, 범위는 500m

map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
#folium.CircleMarker([center_point[0], center_point[1]], radius=100, color="skyblue", fill_color="skyblue").add_to(map)
for i in range(len(centroid)):
    folium.Marker([centroid[i][0], centroid[i][1]], popup=i).add_to(map)
    folium.Circle([centroid[i][0], centroid[i][1]], radius=500, color="skyblue", fill_color="skyblue").add_to(map)
    #folium.PolyLine(locations=[coordinate[i], center_point],weight=2, color = 'blue').add_to(map)/
map

In [9]:
#구별 인덱스 확인

rng=[]
for i in range(25):
    gu=df3["구"].unique()[i]
    rng.append((df3[df3["구"]==gu].index.min(), df3[df3["구"]==gu].index.max()))

rng

[(0, 194),
 (195, 384),
 (385, 567),
 (568, 706),
 (707, 893),
 (894, 1072),
 (1073, 1193),
 (1194, 1334),
 (1335, 1495),
 (1496, 1668),
 (1669, 1855),
 (1856, 2027),
 (2028, 2214),
 (2215, 2385),
 (2386, 2561),
 (2562, 2737),
 (2738, 2922),
 (2923, 3088),
 (3089, 3228),
 (3229, 3377),
 (3378, 3544),
 (3545, 3706),
 (3707, 3876),
 (3877, 4036),
 (4037, 4197)]

In [10]:
# 구 하나
## 강남구

map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
#folium.CircleMarker([center_point[0], center_point[1]], radius=100, color="skyblue", fill_color="skyblue").add_to(map)
for i in range(len(centroid)):
    folium.Marker([centroid[i][0], centroid[i][1]], icon=folium.Icon(color="red")).add_to(map)
    folium.Circle([centroid[i][0], centroid[i][1]], radius=500, color="skyblue", fill_color="skyblue").add_to(map)
    #folium.PolyLine(locations=[coordinate[i], center_point],weight=2, color = 'blue').add_to(map)/

#for i in range(len(coords)):
for i in range(0, 194):
     folium.Marker([coords[i][0], coords[i][1]]).add_to(map)
map


In [11]:
# 각 구별로 찍어보고 저장

for j in range(25):
    map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
    for i in range(len(centroid)):
        folium.Marker([centroid[i][0], centroid[i][1]], icon=folium.Icon(color="red")).add_to(map)
        folium.Circle([centroid[i][0], centroid[i][1]], radius=500, color="skyblue", fill_color="skyblue").add_to(map)
        #folium.PolyLine(locations=[coordinate[i], center_point],weight=2, color = 'blue').add_to(map)/

    #for i in range(len(coords)):
    for k in range(rng[j][0], rng[j][1]+1):
         folium.Marker([coords[k][0], coords[k][1]], popup=k).add_to(map)
    map.save("{}{}{}.html".format(j, " - ", df3["구"].unique()[j]))


In [15]:
# 전체 다 찍어서 저장

map=folium.Map(location=[centroid[0][0], centroid[0][1]], zoom_start=12)
for i in range(len(centroid)):
    folium.Marker([centroid[i][0], centroid[i][1]], icon=folium.Icon(color="red")).add_to(map)
    folium.Circle([centroid[i][0], centroid[i][1]], radius=500, color="skyblue", fill_color="skyblue").add_to(map)

for k in range(4198):
    folium.Marker([coords[k][0], coords[k][1]], popup=k).add_to(map)

map.save("좌표 및 clustering.html")


In [13]:
len(coords)

4198

In [21]:
pd.DataFrame(centroid).to_csv("data/cluster 중간지점 좌표모음.csv", sep=",", index=False)